In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency

In [3]:
df = pd.read_csv('sales_data.csv', parse_dates=['Date'])